In [2]:
#@title Imports and Data
!pip -q install streamlit > /dev/null
!pip -q install pyngrok > /dev/null
%tensorflow_version 2.x
%pip install -q transformers

import cv2
import plotly.express as px
import tensorflow as tf
from urllib.request import urlretrieve
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from transformers import BertTokenizer
from transformers import TFBertModel
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy

model_name = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/train'
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/valid'
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/test'
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/vocab.intent'
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/vocab.slot'

def parse_line(line):
    data, intent_label = line.split(" <=> ")
    items = data.split()
    words = [item.rsplit(":", 1)[0]for item in items]
    word_labels = [item.rsplit(":", 1)[1]for item in items]
    return {
        "intent_label": intent_label, 
        "words": " ".join(words),
        "word_labels": " ".join(word_labels),
        "length": len(words),
    }

def encode_dataset(text_sequences):
    # Create token_ids array (initialized to all zeros), where 
    # rows are a sequence and columns are encoding ids
    # of each token in given sequence.
    token_ids = np.zeros(shape=(len(text_sequences), max_token_len),
                         dtype=np.int32)
    
    for i, text_sequence in enumerate(text_sequences):
        encoded = tokenizer.encode(text_sequence)
        token_ids[i, 0:len(encoded)] = encoded

    attention_masks = (token_ids != 0).astype(np.int32)
    return {"input_ids": token_ids, "attention_masks": attention_masks}


train_lines = Path("train").read_text().strip().splitlines()
valid_lines = Path("valid").read_text().strip().splitlines()
test_lines = Path("test").read_text().strip().splitlines()

df_train = pd.DataFrame([parse_line(line) for line in train_lines])
df_valid = pd.DataFrame([parse_line(line) for line in valid_lines])
df_test = pd.DataFrame([parse_line(line) for line in test_lines])

max_token_len = 43

encoded_train = encode_dataset(df_train["words"])
encoded_valid = encode_dataset(df_valid["words"])
encoded_test = encode_dataset(df_test["words"])

intent_names = Path("vocab.intent").read_text().split()
intent_map = dict((label, idx) for idx, label in enumerate(intent_names))
intent_train = df_train["intent_label"].map(intent_map).values
intent_valid = df_valid["intent_label"].map(intent_map).values
intent_test = df_test["intent_label"].map(intent_map).values

base_bert_model = TFBertModel.from_pretrained("bert-base-cased")

ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.19 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 6.0.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.25.0 which is incompatible.
     |████████████████████████████████| 2.5MB 24.5MB/s 
     |████████████████████████████████| 901kB 46.3MB/s 
     |████████████████████████████████| 3.3MB 38.7MB/s 



--2021-07-16 15:04:18--  https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/train
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.128, 172.217.2.112, 172.217.12.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1793794 (1.7M) [application/octet-stream]
Saving to: ‘train’

train               100%[===================>]   1.71M  --.-KB/s    in 0.01s   

2021-07-16 15:04:18 (177 MB/s) - ‘train’ saved [1793794/1793794]

--2021-07-16 15:04:18--  https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/valid
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.9.208, 172.253.115.128, 172.253.122.128, ...
Connecting to storage.googleapis.co

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [26]:
# Build a map from slot name to a unique id.
slot_names = ["[PAD]"] + Path("vocab.slot").read_text().strip().splitlines()
slot_map = {}
for label in slot_names:
    slot_map[label] = len(slot_map)
slot_map

{'B-album': 1,
 'B-artist': 2,
 'B-best_rating': 3,
 'B-city': 4,
 'B-condition_description': 5,
 'B-condition_temperature': 6,
 'B-country': 7,
 'B-cuisine': 8,
 'B-current_location': 9,
 'B-entity_name': 10,
 'B-facility': 11,
 'B-genre': 12,
 'B-geographic_poi': 13,
 'B-location_name': 14,
 'B-movie_name': 15,
 'B-movie_type': 16,
 'B-music_item': 17,
 'B-object_location_type': 18,
 'B-object_name': 19,
 'B-object_part_of_series_type': 20,
 'B-object_select': 21,
 'B-object_type': 22,
 'B-party_size_description': 23,
 'B-party_size_number': 24,
 'B-playlist': 25,
 'B-playlist_owner': 26,
 'B-poi': 27,
 'B-rating_unit': 28,
 'B-rating_value': 29,
 'B-restaurant_name': 30,
 'B-restaurant_type': 31,
 'B-served_dish': 32,
 'B-service': 33,
 'B-sort': 34,
 'B-spatial_relation': 35,
 'B-state': 36,
 'B-timeRange': 37,
 'B-track': 38,
 'B-year': 39,
 'I-album': 40,
 'I-artist': 41,
 'I-city': 42,
 'I-country': 43,
 'I-cuisine': 44,
 'I-current_location': 45,
 'I-entity_name': 46,
 'I-facil

In [4]:
# Uses the slot_map of slot name to unique id, defined above, as well
# as the BERT tokenizer, to create a np array with each row corresponding
# to a given sequence, and the columns as the id of the given token slot labels.
def encode_token_labels(text_sequences, true_word_labels):
    encoded = np.zeros(shape=(len(text_sequences), max_token_len), dtype=np.int32)
    
    # Loop through sequence
    for i, (text_sequence, word_labels) in enumerate( \
            zip(text_sequences, true_word_labels)):
        encoded_labels = []
        
        # Loop through word in sequence
        for word, word_label in zip(text_sequence.split(), word_labels.split()):
            tokens = tokenizer.tokenize(word)
            encoded_labels.append(slot_map[word_label])
            expand_label = word_label.replace("B-", "I-")
            if not expand_label in slot_map:
                expand_label = word_label
            encoded_labels.extend([slot_map[expand_label]] * (len(tokens) - 1))
        encoded[i, 1:len(encoded_labels) + 1] = encoded_labels
    return encoded

In [5]:
# Encode the token labels and store in variables slot_train, slot_valid, slot_test.
slot_train = encode_token_labels(df_train['words'], df_train['word_labels'])
slot_valid = encode_token_labels(df_valid['words'], df_train['word_labels'])
slot_test = encode_token_labels(df_test['words'], df_train['word_labels'])

In [6]:
import tensorflow as tf
#from mrcnn import config as config_std
# Define the class for the model that will create predictions
# for the overall intent of a sequence, as well as the NER token labels.
class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_num_labels=None, slot_num_labels=None,
                dropout_prob=0.4):
        super().__init__(name="joint_intent_slot")

        self.bert = base_bert_model
        
        self.dropout = Dropout(dropout_prob)
        self.intent_classifier = Dense(intent_num_labels,
                                       name="intent_classifier")
        self.slot_classifier = Dense(slot_num_labels,
                                     name="slot_classifier")

    def call(self, inputs, **kwargs):
        tokens_output, pooled_output = self.bert(inputs, **kwargs, return_dict=False)

        tokens_output = self.dropout(tokens_output, training=kwargs.get("training", False))
        slot_logits = self.slot_classifier(tokens_output)

        pooled_output = self.dropout(pooled_output, training=kwargs.get("training", False))
        intent_logits = self.intent_classifier(pooled_output)

        return slot_logits, intent_logits

joint_model = JointIntentAndSlotFillingModel(intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))

# Define one classification loss for each output (intent & NER):
losses = [SparseCategoricalCrossentropy(from_logits=True),
          SparseCategoricalCrossentropy(from_logits=True)]
          
joint_model.compile(optimizer=Adam(learning_rate=3e-5, epsilon=1e-08),
                    loss=losses,
                    metrics=[SparseCategoricalAccuracy('accuracy')], run_eagerly=True)

# Train the model.
history = joint_model.fit(encoded_train, (slot_train, intent_train), \
    validation_data=(encoded_valid, (slot_valid, intent_valid)), \
    epochs=3, batch_size=32)
joint_model.summary()

Epoch 1/3
409/409 [==============================] - 174s 425ms/step - loss: 0.5882 - output_1_loss: 0.3647 - output_2_loss: 0.2235 - output_1_accuracy: 0.9183 - output_2_accuracy: 0.9266 - val_loss: 2.1446 - val_output_1_loss: 2.0532 - val_output_2_loss: 0.0914 - val_output_1_accuracy: 0.7925 - val_output_2_accuracy: 0.9800
Epoch 2/3
409/409 [==============================] - 176s 431ms/step - loss: 0.0981 - output_1_loss: 0.0601 - output_2_loss: 0.0380 - output_1_accuracy: 0.9840 - output_2_accuracy: 0.9891 - val_loss: 2.5702 - val_output_1_loss: 2.5210 - val_output_2_loss: 0.0492 - val_output_1_accuracy: 0.7920 - val_output_2_accuracy: 0.9871
Epoch 3/3
409/409 [==============================] - 176s 432ms/step - loss: 0.0557 - output_1_loss: 0.0325 - output_2_loss: 0.0231 - output_1_accuracy: 0.9910 - output_2_accuracy: 0.9938 - val_loss: 2.7287 - val_output_1_loss: 2.6613 - val_output_2_loss: 0.0675 - val_output_1_accuracy: 0.7921 - val_output_2_accuracy: 0.9829
Model: "joint_inten

In [7]:
joint_model.save_weights('./weights')

In [8]:
df_train.head()

,intent_label,words,word_labels,length
0,AddToPlaylist,Add Don and Sherri to my Meditate to Sounds of...,O B-entity_name I-entity_name I-entity_name O ...,12
1,AddToPlaylist,put United Abominations onto my rare groove pl...,O B-entity_name I-entity_name O B-playlist_own...,8
2,AddToPlaylist,add the tune by misato watanabe to the Trapeo ...,O O B-music_item O B-artist I-artist O O B-pla...,10
3,AddToPlaylist,add this artist to my this is miguel bosé play...,O O B-music_item O B-playlist_owner B-playlist...,10
4,AddToPlaylist,add heresy and the hotel choir to the evening ...,O B-entity_name I-entity_name I-entity_name I-...,11


In [9]:
def show_predictions(text, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = joint_model(inputs)
    slot_logits, intent_logits = outputs
    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]
    print("## Intent:", intent_names[intent_id])
    print("## Slots:")
    for token, slot_id in zip(tokenizer.tokenize(text), slot_ids):
        print(f"{token:>10} : {slot_names[slot_id]}")

show_predictions("Add a song to my playlist Good Songs", intent_names, slot_names)

## Intent: AddToPlaylist
## Slots:
        Ad : O
       ##d : O
         a : O
      song : B-music_item
        to : O
        my : B-playlist_owner
      play : O
    ##list : O
      Good : B-playlist
     Songs : I-playlist


In [31]:
#@title Run this cell to initialize your website!
%%writefile app.py
import streamlit as st

import cv2
import plotly.express as px
import tensorflow as tf
from urllib.request import urlretrieve
from pathlib import Path
import matplotlib.pyplot as plt
import altair as alt
import numpy as np
import pandas as pd
from transformers import BertTokenizer
from transformers import TFBertModel
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy

model_name = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

def parse_line(line):
    data, intent_label = line.split(" <=> ")
    items = data.split()
    words = [item.rsplit(":", 1)[0]for item in items]
    word_labels = [item.rsplit(":", 1)[1]for item in items]
    return {
        "intent_label": intent_label, 
        "words": " ".join(words),
        "word_labels": " ".join(word_labels),
        "length": len(words),
    }

def encode_dataset(text_sequences):
    # Create token_ids array (initialized to all zeros), where 
    # rows are a sequence and columns are encoding ids
    # of each token in given sequence.
    token_ids = np.zeros(shape=(len(text_sequences), max_token_len),
                         dtype=np.int32)
    
    for i, text_sequence in enumerate(text_sequences):
        encoded = tokenizer.encode(text_sequence)
        token_ids[i, 0:len(encoded)] = encoded

    attention_masks = (token_ids != 0).astype(np.int32)
    return {"input_ids": token_ids, "attention_masks": attention_masks}

train_lines = Path("train").read_text().strip().splitlines()
valid_lines = Path("valid").read_text().strip().splitlines()
test_lines = Path("test").read_text().strip().splitlines()

df_train = pd.DataFrame([parse_line(line) for line in train_lines])
df_valid = pd.DataFrame([parse_line(line) for line in valid_lines])
df_test = pd.DataFrame([parse_line(line) for line in test_lines])

max_token_len = 43

encoded_train = encode_dataset(df_train["words"])
encoded_valid = encode_dataset(df_valid["words"])
encoded_test = encode_dataset(df_test["words"])

intent_names = Path("vocab.intent").read_text().split()
intent_map = dict((label, idx) for idx, label in enumerate(intent_names))
intent_train = df_train["intent_label"].map(intent_map).values
intent_valid = df_valid["intent_label"].map(intent_map).values
intent_test = df_test["intent_label"].map(intent_map).values

base_bert_model = TFBertModel.from_pretrained("bert-base-cased")


# Build a map from slot name to a unique id.
slot_names = ["[PAD]"] + Path("vocab.slot").read_text().strip().splitlines()
slot_map = {}
for label in slot_names:
    slot_map[label] = len(slot_map)

# Uses the slot_map of slot name to unique id, defined above, as well
# as the BERT tokenizer, to create a np array with each row corresponding
# to a given sequence, and the columns as the id of the given token slot labels.
def encode_token_labels(text_sequences, true_word_labels):
    encoded = np.zeros(shape=(len(text_sequences), max_token_len), dtype=np.int32)
    
    # Loop through sequence
    for i, (text_sequence, word_labels) in enumerate( \
            zip(text_sequences, true_word_labels)):
        encoded_labels = []
        
        # Loop through word in sequence
        for word, word_label in zip(text_sequence.split(), word_labels.split()):
            tokens = tokenizer.tokenize(word)
            encoded_labels.append(slot_map[word_label])
            expand_label = word_label.replace("B-", "I-")
            if not expand_label in slot_map:
                expand_label = word_label
            encoded_labels.extend([slot_map[expand_label]] * (len(tokens) - 1))
        encoded[i, 1:len(encoded_labels) + 1] = encoded_labels
    return encoded

#from mrcnn import config as config_std
# Define the class for the model that will create predictions
# for the overall intent of a sequence, as well as the NER token labels.
class JointIntentAndSlotFillingModel(tf.keras.Model):
    def __init__(self, intent_num_labels=None, slot_num_labels=None,
                dropout_prob=0.4):
        super().__init__(name="joint_intent_slot")

        self.bert = base_bert_model
        
        self.dropout = Dropout(dropout_prob)
        self.intent_classifier = Dense(intent_num_labels,
                                       name="intent_classifier")
        self.slot_classifier = Dense(slot_num_labels,
                                     name="slot_classifier")

    def call(self, inputs, **kwargs):
        tokens_output, pooled_output = self.bert(inputs, **kwargs, return_dict=False)

        tokens_output = self.dropout(tokens_output, training=kwargs.get("training", False))
        slot_logits = self.slot_classifier(tokens_output)

        pooled_output = self.dropout(pooled_output, training=kwargs.get("training", False))
        intent_logits = self.intent_classifier(pooled_output)

        return slot_logits, intent_logits

joint_model = JointIntentAndSlotFillingModel(7, 73)
joint_model.load_weights('weights')


def show_predictions(text, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = joint_model(inputs)
    slot_logits, intent_logits = outputs
    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]
    st.text("## Intent:" + str(intent_names[intent_id]))
    st.text("## Slots:")
    for token, slot_id in zip(tokenizer.tokenize(text), slot_ids):
        st.text("   " + f"{token:>10} : {slot_names[slot_id]}")


options = ["Demonstration", "Data", "Sequence Intents", "Slot Intents"]
selectbox = st.sidebar.selectbox(
    "What would you like to see?",
    (option for option in options)
)

if selectbox == options[0]:
  selectbox2 = None
  st.title("Demonstration")
  st.header("Type your input below.")
  user_input = st.text_input("", "")
  if user_input is not "":
    show_predictions(user_input, intent_names, slot_names)
  
  st.header("**What you can ask me:**")
  st.subheader("Add to a playlist")
  st.text("Add ___ to the playlist ___.")
  st.subheader("Book a restaurant")
  st.text("Book a table for tonight at ___.")
  st.subheader("Play music")
  st.text("Play ___. ")
  st.subheader("Rate a book")
  st.text("Rate ___ a zero out of ten.")
  st.subheader("Get weather")
  st.text("What's the weather in ___.")
  st.subheader("Search a creative work")
  st.text("Show me ___.")
  st.subheader("Search a screening event")
  st.text("Find the closest showing of ___.")
elif selectbox == options[1]:
  options2 = ["Training", "Validation", "Testing"]
  selectbox2 = st.sidebar.selectbox(
    "Which data set would you like to see?",
    (option for option in options2)
  )
  if selectbox2 == options2[0]:
    st.title("**Training Data:**")
    st.dataframe(df_train)

  elif selectbox2 == options2[1]:
    st.title("**Validation Data:**")
    st.dataframe(df_valid)

  elif selectbox2 == options2[2]:
    st.title("**Testing Data:**")
    st.dataframe(df_test)
elif selectbox == options[2]:
  st.title("**Sequence Intents:**")
  st.dataframe(pd.DataFrame(intent_names))
elif selectbox == options[3]:
  st.title("**Slot Intents:**")
  st.dataframe(pd.DataFrame([intent for intent in slot_map]), width = 600)
  




Overwriting app.py


In [32]:
from pyngrok import ngrok
#Publish Web App (Run this again whenever you make changes)
public_url = ngrok.connect(port='80')
print (public_url)
! streamlit run --server.port 80 app.py

Streaming output truncated to the last 5000 lines.
2021-07-16 16:56:29.908 Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).bert.bert.encoder.layer.3.bert_output.LayerNorm.gamma
2021-07-16 16:56:29.909 Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).bert.bert.encoder.layer.3.bert_output.LayerNorm.beta
2021-07-16 16:56:29.909 Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).bert.bert.encoder.layer.4.intermediate.dense.kernel
2021-07-16 16:56:29.909 Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).bert.bert.encoder.layer.4.intermediate.dense.bias
2021-07-16 16:56:29.910 Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).bert.bert.encoder.layer.4.bert_output.dense.kernel
2021-07-16 16:56:29.912 Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).bert.bert.encoder.layer.4.bert_output.dense.bias
2021-07-16 16:56:29.912 Unresolved object in checkpoint: (